In [6]:
import openai
from langchain import SQLDatabaseChain
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from sqlalchemy.engine import URL
from langchain.chains import SQLDatabaseSequentialChain
def connect_to_DB():
    driver = '{SQL Server}'
    server = 'Your Server'
    database = 'NorthWind'

    # pyodbc connection string
    connection_string = f'DRIVER={driver};SERVER={server};'
    connection_string += f'DATABASE={database};'

    connection_url = URL.create(
        "mssql+pyodbc", query={"odbc_connect": connection_string})
    db = SQLDatabase.from_uri(connection_url, engine_args={"use_setinputsizes":False})

    return db

In [ ]:


OPENAI_API_KEY = 'Your Key Here'
engine = connect_to_DB()
llm = OpenAI(temperature=0, verbose=False, openai_api_key=OPENAI_API_KEY, model_name='gpt-3.5-turbo-16k-0613')
db_chain = SQLDatabaseSequentialChain.from_llm(llm, engine, verbose=True)#, return_intermediate_steps=True

In [0]:
# while True:
#     query = input()
#     print(query)
#     try:
#         result = db_chain.run(query)
#         i = 0
#         #result["intermediate_steps"]
#     except openai.error.InvalidRequestError:
#         continue

In [10]:
query = 'Which Employee has the most orders'#2 Tables with top
print(query)
try:
    result = db_chain.run(query)
    i = 0
    #result["intermediate_steps"]
except openai.error.InvalidRequestError:
    print('Invalid Query')

Which Employee has the most orders


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['Employees', 'Orders']

> Entering new SQLDatabaseChain chain...
Which Employee has the most orders
SQLQuery:SELECT TOP 1 e.EmployeeID, e.LastName, e.FirstName, COUNT(o.OrderID) AS OrderCount
FROM Employees e
JOIN Orders o ON e.EmployeeID = o.EmployeeID
GROUP BY e.EmployeeID, e.LastName, e.FirstName
ORDER BY OrderCount DESC
SQLResult: [(4, 'Peacock', 'Margaret', 156)]
Answer:Margaret Peacock has the most orders with a total of 156 orders.
> Finished chain.

> Finished chain.


In [13]:
query = 'Which employee sold the most products'#3 Tables with Top
print(query)
try:
    result = db_chain.run(query)
    i = 0
    #result["intermediate_steps"]
except openai.error.InvalidRequestError:
    print('Invalid Query')

Which employee sold the most products


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['Employees', 'Order Details', 'Products']

> Entering new SQLDatabaseChain chain...
Which employee sold the most products
SQLQuery:SELECT TOP 1 e.FirstName, e.LastName, COUNT(od.ProductID) AS TotalProductsSold
FROM Employees e
JOIN Orders o ON e.EmployeeID = o.EmployeeID
JOIN [Order Details] od ON o.OrderID = od.OrderID
GROUP BY e.FirstName, e.LastName
ORDER BY TotalProductsSold DESC
SQLResult: [('Margaret', 'Peacock', 420)]
Answer:Margaret Peacock sold the most products.
> Finished chain.

> Finished chain.


In [17]:
query = 'How much money did we make this year'# It fails with a null and should put in ifnull()
print(query)
try:
    result = db_chain.run(query)
    i = 0
    #result["intermediate_steps"]
except openai.error.InvalidRequestError:
    print('Invalid Query')

How much money did we make this year


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['Orders', 'Order Details']

> Entering new SQLDatabaseChain chain...
How much money did we make this year
SQLQuery:SELECT SUM(UnitPrice * Quantity) AS TotalRevenue
FROM [Order Details]
JOIN [Orders] ON [Order Details].OrderID = [Orders].OrderID
WHERE YEAR([Orders].OrderDate) = YEAR(GETDATE())
SQLResult: [(None,)]
Answer:Final answer here: The company did not make any money this year.
> Finished chain.

> Finished chain.


In [18]:
query = 'Who services region 2'#A 3 table join and filters the doubles
print(query)
try:
    result = db_chain.run(query)
    i = 0
    #result["intermediate_steps"]
except openai.error.InvalidRequestError:
    print('Invalid Query')

Who services region 2


> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['Territories', 'Employees', 'EmployeeTerritories']

> Entering new SQLDatabaseChain chain...
Who services region 2
SQLQuery:SELECT e.FirstName, e.LastName
FROM Employees e
JOIN EmployeeTerritories et ON e.EmployeeID = et.EmployeeID
JOIN Territories t ON et.TerritoryID = t.TerritoryID
WHERE t.RegionID = 2
SQLResult: [('Michael', 'Suyama'), ('Michael', 'Suyama'), ('Michael', 'Suyama'), ('Michael', 'Suyama'), ('Michael', 'Suyama'), ('Robert', 'King'), ('Robert', 'King'), ('Robert', 'King'), ('Robert', 'King'), ('Robert', 'King'), ('Robert', 'King'), ('Robert', 'King'), ('Robert', 'King'), ('Robert', 'King'), ('Robert', 'King')]
Answer:Final answer: Michael Suyama and Robert King service region 2.
> Finished chain.

> Finished chain.
